# Agent

Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

代理是使用大型語言模型（LLMs）作為推理引擎來決定採取哪些行動以及傳遞哪些輸入的系統。執行行動後，結果可以回饋到大型語言模型中，以確定是否需要更多的行動，或者是否可以結束。


- Based on the prompt, Agent creates an LLM based 'Thought': what it should do
- Based on the 'Thought', Angent takes 'Action'.

## Agent Tools:

- API services: OpenAI Whisper-1, TTS-1, DALLE3, Google Search
- Precision matters: Mathematics, Scientific Calculation
- Extra knowledge: Wikipedia, Private Dataset

In [ ]:
from IPython.display import Image

Image(url='https://statusneo.com/wp-content/uploads/2024/01/fe9fa1ac-dfde-4d91-8b5b-4497b742c414_1400x686.jpg')

In [ ]:
import os

os.chdir("../../")

## Model + Tools

Tool calling allows a chat model to respond to a given prompt by "calling a tool". While the name implies that the model is performing some action, this is actually not the case! The model generates the arguments to a tool, and actually running the tool (or not) is up to the user. For example, if you want to extract output matching some schema from unstructured text, you could give the model an "extraction" tool that takes parameters matching the desired schema, then treat the generated output as your final result.

In [ ]:
from langchain.chat_models import ChatOpenAI
from src.initialization import credential_init

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

## Zero-Shot Agent

- PromptTemplate.from_template: This creates a template for the AI to follow when generating responses. A "zero-shot" prompt means the AI will generate answers without needing specific examples beforehand.

- create_react_agent: This function creates an AI agent using a specific language model (LLM). The agent can react to prompts based on the template provided.

- AgentExecutor: This sets up an executor that can run the agent. The verbose=True part means it will provide detailed output about what it’s doing.

- agent_executor.invoke: This runs the agent with the given input. In this case, it's asking the agent to calculate the area of a circle with a specified radius.

- PromptTemplate.from_template: 創建一個模板，供AI在生成回應時遵循。"zero-shot" 提示意味著AI在生成回答時不需要事先的具體示例。

- create_react_agent: 該函數使用特定的語言模型（LLM）創建一個AI代理。該代理可以根據提供的模板對提示做出反應。

- AgentExecutor: 設置一個執行器來運行代理。verbose=True部分意味著它會提供詳細的輸出。

- agent_executor.invoke: 這會運行具有給定輸入的代理。在此情況下，它是請求代理計算具有指定半徑的圓的面積。

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

from src.agent.react_zero_shot import prompt_template as zero_shot_prompt_template

prompt = PromptTemplate.from_template(zero_shot_prompt_template)

zero_shot_agent = create_react_agent(
    llm=model,
    tools=[],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=[], verbose=True)

In [ ]:
agent_executor.invoke({"input": "can you calculate the area of a circle that has a radius of 10.923mm"})

In [ ]:
import numpy as np

10.923 **2 * np.pi

### CircleAreaTool Class:

    - name: This sets the name of the tool to "Circle area calculator".
    - description: This provides a description of when to use the tool.
    - _run Method: This method performs the calculation of the area using the radius provided. It converts the radius to a float and calculates the area using the formula 

$$ \text{Area} = \pi \times \text{r}^2 $$

    - _arun Method: This raises an error because asynchronous operation is not supported by this tool.

In [ ]:
from math import pi
from typing import Union

from langchain.tools import BaseTool


"""
When defining a tool like this using the BaseTool template we must define `name` and `description` attributes, alongside _run and _arun methods. 
When a tool us used the _run method is called by default. The _arun method is called when the tool is to be used asyncronously. We do not cover 
that in this walkthrough so for now we create it with a NotImplementedError.
"""

class CircleAreaTool(BaseTool):
    name = "Circle area calculator"
    description = 'use this tool when you need to calculate an area using the radius of a circle'
    
    def _run(self, radius: Union[int, float]):
        r = float(radius)
        return  r * r * pi
    
    def _arun(self, radius: Union[int, float]):
        raise NotImplementedError("This tool does not support async")
    

### Tools List:

- This list now includes the CircleAreaTool instance, making it available for the agent to use.

In [ ]:
# Add tools

tools = [CircleAreaTool()]

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "can you calculate the area of a circle that has a radius of 10.923mm"})

## Docstore Agent

A zero shot agent that does a reasoning step before acting.
This agent has access to a document store that allows it to look up relevant information to answering the question.

** To what I understand, it is an Zero Shot Agent with a database query tool.


Docstore Agent 是專門為使用 LangChain docstore 進行信息搜索（Search）和查找（Lookup）而構建的。
Search：從文檔庫中檢索相關的頁面
Lookup：從檢索出的相關頁面中，查找相關的具體內容

### 1. Wikipedia Query Setup (維基百科查詢設置)

- The first line creates a wikipedia object that can run queries on Wikipedia. It uses something called WikipediaAPIWrapper to do this. Imagine this as giving the assistant access to a huge online library (Wikipedia).
- 第一行創建了一個可以在維基百科上運行查詢的 wikipedia 對象。它使用了一個叫做 WikipediaAPIWrapper 的東西。想象一下，這就像是給助手訪問一個巨大的在線圖書館（維基百科）的權限。

In [ ]:
from langchain.agents import Tool
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

### 2. Search Tool Initialization (搜索工具初始化)

- Next, we create a tool called search_tool. This tool uses the wikipedia object to look up information on Wikipedia. Think of this tool as a search button specifically for Wikipedia.

- List of Tools

- 接下來，我們創建一個叫做 search_tool 的工具。這個工具使用 wikipedia 對象在維基百科上查找信息。想象這個工具就像是一個專門用於維基百科的搜索按鈕。

- 工具列表

In [ ]:
# initialize the docstore search tool
search_tool = Tool(
    name="Search Engine Tool",
    func=wikipedia.run,
    description='search wikipedia'
)

tools = [search_tool]

### 3. Creating the Agent (創建代理)

- The next step is to create the intelligent agent using the prompt template and tools we set up. We call this agent docstore_agent. This is like creating the brain of our smart assistant.

- Agent Executor

- 下一步是使用我們設置的提示模板和工具創建智能代理。我們稱這個代理為 docstore_agent。這就像是在創建我們智能助手的大腦。

- 代理執行器

In [ ]:
prompt = PromptTemplate.from_template(zero_shot_prompt_template)

docstore_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=docstore_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "How is the relationship between Ukraine and Russian in 2023?"})

### Can we build our own docstore?

In [ ]:
import pandas as pd
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# https://platform.openai.com/docs/guides/embeddings/what-are-embeddings

# A list of embedding models you can choose 
# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

### Create the retriever

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_core.output_parsers import StrOutputParser

df_cnn = pd.read_csv("tutorial/Week-5/CNN_Articels_clean.csv")

documents = []

for _, row in df_cnn.iloc[:20].iterrows():
    document = Document(page_content=row['Headline'],
                        metadata={'Author': row['Author'],
                                  'Category': row['Category'],
                                  'Section': row['Section']})
    documents.append(document)

vectorstore = FAISS.from_documents(documents, embedding=embedding)

retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

In [ ]:
_TEMPLATE = """Generate a summary based on the context

{context}

Summary:
"""

prompt = PromptTemplate.from_template(_TEMPLATE)

chain = {'context': retriever}|prompt|model|StrOutputParser()

In [ ]:
search_tool = Tool(
    name="Search Engine Tool",
    func=chain.invoke,
    description='Search CNN news'
)

tools = [search_tool]

prompt = PromptTemplate.from_template(zero_shot_prompt_template)

docstore_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=docstore_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "How is the current situation of truck driver in the US?"})

## Conversational Agent

In [ ]:
from src.agent.react_chat import prompt_template as chat_prompt_template

prompt = PromptTemplate.from_template(chat_prompt_template)

conversation_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=conversation_agent, tools=tools, verbose=True)

In [ ]:
# In week-4, we had 

from langchain.memory import ChatMessageHistory

demo_chat_history = ChatMessageHistory()

demo_chat_history.add_user_message("hi!")

demo_chat_history.add_ai_message("whats up?")

demo_chat_history.messages

In [ ]:
chat_history = [(message.type, message.content) for message in demo_chat_history.messages]

In [ ]:
agent_executor.invoke({"chat_history": chat_history,
                       "input": "can you calculate the area of a circle that has a radius of 10.923mm"})

In [ ]:
"""
聊天Agent邏輯:

for query in queries:
   res = agent_executor.invoke({"chat_history": chat_history,
                                "input": query})
   chat_history.append(('human', query))
   chat_history.append(('ai', res['output']))
"""

## self ask with search agent

For this agent, only one tool can be used and it needs to be named "Intermediate Answer"

prompt: hwchase17/self-ask-with-search

It seems it does not work well with OpenAI.

### Google Search

- https://serper.dev/
- 2500 free queries

In [ ]:
from langchain.agents import create_self_ask_with_search_agent

"""
An agent that breaks down a complex question into a series of simpler questions.

This agent uses a search tool to look up answers to the simpler questions in order to answer the original complex question.
"""

In [ ]:
import os

from langchain.agents import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI

llm = OpenAI()

search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )]


In [ ]:
prompt_template = """
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

Question: Are both the directors of Jaws and Casino Royale from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate answer: New Zealand.
So the final answer is: No

Question: {input}

Are followup questions needed here:{agent_scratchpad}
"""


prompt = PromptTemplate.from_template(prompt_template)

self_ask_with_search_agent = create_self_ask_with_search_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=self_ask_with_search_agent, tools=tools, verbose=True)

# agent_executor.invoke({"input": "What is the hometown of the reigning men's 2022 U.S. Open champion?"}, return_only_outputs=True)

### GPT-4o can achieve the same functionality with zero-shot.

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent


search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Google Search",
        func=search.run,
        description="useful for when you need to search",
    )]


prompt = PromptTemplate.from_template(zero_shot_prompt_template)

zero_shot_agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=zero_shot_agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "What is the hometown of the reigning men's 2022 U.S. Open champion?"}, return_only_outputs=True)

## Structured Chat Agent

The structured chat agent is capable of using multi-input tools.

在 0.1.0 版本之前，如果 Agent 用到的工具的輸入參數如果不是 1 個的話，那麼就只能是用 Structured Chat Agent。但是 0.1.0 版本取消了這個限制。因此，
在 0.1.0 版本之後，Structured Chat Agent 和 ReAct Agent 的主要區別就只剩下工具的描述方式：

Structured Chat Agent 需要使用 JSON-Schema 的模式來創建結構化的參數輸入，對於更複雜的工具而言，這種方式更為有用。

- https://smith.langchain.com/hub/hwchase17/structured-chat-agent?organizationId=c4887cc4-1275-5361-82f2-b22aee75bad1

In [ ]:
from typing import Optional

import numpy as np
from langchain.agents import create_structured_chat_agent
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate

from src.agent.structured_react_chat import system_message_template, human_message_template

messages = [
    SystemMessagePromptTemplate.from_template(system_message_template),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template(human_message_template),
]

input_variables = ["tools", "tool_names", "input", "chat_history", "agent_scratchpad"]

prompt = ChatPromptTemplate(input_variables=input_variables, messages=messages)


class PythagorasTool(BaseTool):
    name = "Hypotenuse calculator"
    description = """
    Use this tool when you need to calculate the length of an hypotenuse given one or two sides of a triangle and/or an angle (in degrees).
    To use the tool you must provide at least two of the following parameters ['adjacent_side', 'opposite_side', 'angle'].
    """

    def _run(
        self,
        adjacent_side: Optional[Union[int, float]] = None,
        opposite_side: Optional[Union[int, float]] = None,
        angle: Optional[Union[int, float]] = None
    ):
        # check for the values we have been given
        if adjacent_side and opposite_side:
            return np.sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")

tools = [PythagorasTool()]

struct_agent = create_structured_chat_agent(
    llm=model,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=struct_agent, tools=tools, verbose=True)

In [ ]:
from langchain.memory import ChatMessageHistory

demo_chat_history = ChatMessageHistory()

demo_chat_history.add_user_message("hi!")

demo_chat_history.add_ai_message("whats up?")

agent_executor.invoke({"input": "If I have a triangle with the opposite side of length 51 and the adjacent side of 40, what is the length of the hypotenuse?", 
                       "chat_history": demo_chat_history.messages})

In [ ]:
import numpy as np

np.sqrt(51**2 + 40**2)

## 回家作業

- 在Zero Shot Agent 中，建立一個給予兩邊長和夾角，計算三角形面積的功能 
- 在Agent的工具欄中，同時放入Google Search 和 Image Caption 工具